In [1]:
import logging
import sys
import nest_asyncio

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

nest_asyncio.apply()

In [2]:
from llama_index import (
    VectorStoreIndex,
    ServiceContext,
    Response,
    
)
import pinecone
import openai
import streamlit as st
from llama_index.vector_stores import PineconeVectorStore
from llama_index.llms import OpenAI
from llama_index.evaluation import RelevancyEvaluator
import pandas as pd

pd.set_option("display.max_colwidth", 0)

INFO:numexpr.utils:NumExpr defaulting to 8 threads.
NumExpr defaulting to 8 threads.


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pinecone/index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
openai.api_key = st.secrets.openai_key
api_key = "2bcecfc8-35f9-44dc-9459-ab0cc2781b29"
pinecone.init(api_key=api_key, environment="gcp-starter")
pinecone_index = pinecone.Index("tailwind-ada")

In [4]:
pinecone_index = pinecone.Index("tailwind-ada")
vector_store = PineconeVectorStore(pinecone_index=pinecone_index)
service_context = ServiceContext.from_defaults(llm=OpenAI(model="gpt-3.5-turbo", temperature=0.5, system_prompt="You are an expert on TailwindCSS and your job is to answer technical questions. Assume that all questions are related to TailwindCSS. Keep your answers technical and based on facts – do not hallucinate features."))
index = VectorStoreIndex.from_vector_store(vector_store=vector_store, service_context=service_context)

In [5]:
evaluator = RelevancyEvaluator(service_context=service_context)

In [6]:
def display_eval_df(query: str, response: Response, eval_result: str) -> None:
    eval_df = pd.DataFrame(
        {
            "Query": query,
            "Response": str(response),
            "Source": response.source_nodes[0].node.text[:1000] + "...",
            "Evaluation Result": "Pass" if eval_result.passing else "Fail",
        },
        index=[0],
    )
    eval_df = eval_df.style.set_properties(
        **{
            "inline-size": "600px",
            "overflow-wrap": "break-word",
        },
        subset=["Response", "Source"]
    )
    display(eval_df)

In [7]:
query_str = "What is border-radius? And how do I use it in TailwindCSS?"
query_engine = index.as_query_engine()
response_vector = query_engine.query(query_str)
eval_result_1 = evaluator.evaluate_response(
    query=query_str, response=response_vector
)


In [8]:
display_eval_df(query_str, response_vector, eval_result_1)

,Query,Response,Source,Evaluation Result
0,What is border-radius? And how do I use it in TailwindCSS?,"Border-radius is a CSS property that allows you to control the roundness of the corners of an element. In TailwindCSS, you can use the border-radius utility classes to apply different roundness values to elements. To use the border-radius utility classes in TailwindCSS, you can apply them directly to the desired element. For example, you can add the class ""rounded"" to apply a default border radius, or use the class ""rounded-lg"" to apply a larger border radius. If you want to customize the default border radius values in TailwindCSS, you can edit the `theme.borderRadius` section of your Tailwind config file. By default, Tailwind provides five border radius size utilities, but you can change, add, or remove these as per your requirements. For example, in the `theme.borderRadius` section of your Tailwind config file, you can specify the desired values for different border radius sizes. You can use keywords like ""none"", ""sm"", ""md"", ""lg"", and ""full"" to specify different sizes, and assign corresponding pixel or rem values to them. Once you have customized the border radius values in your Tailwind config file, you can use the updated values by applying the corresponding utility classes to your elements. Remember that you can also use arbitrary values for border radius by using the `rounded-[value]` class. For example, you can use `rounded-[12px]` to apply a specific border radius of 12 pixels to an element.","--- ## Applying conditionally ### Hover, focus, and other states ### Breakpoints and media queries --- ## Using custom values ### Customizing your theme By default, Tailwind provides five border radius size utilities. You can change, add, or remove these by editing the `theme.borderRadius` section of your Tailwind config. ```diff-js {{ filename: 'tailwind.config.js' }} module.exports = { theme: { borderRadius: { 'none': '0', - 'sm': '0.125rem', - DEFAULT: '0.25rem', + DEFAULT: '4px', - 'md': '0.375rem', - 'lg': '0.5rem', - 'full': '9999px', + 'large': '12px', } } } ``` Learn more about customizing the default theme in the [theme customization](/docs/theme#customizing-t...",Pass
